<a href="https://colab.research.google.com/github/sreeraksha-ss/CNN-Projects/blob/main/VGG16_Chest_disease(Multilabel_CNN_project).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

VGG16

VGG16 is a convolutional neural network architecture that has been widely used for image classification tasks.
This project  aims to detect and classify chest X-rays based on two diseases (Pnuemonia and covid-19 Xrays ) with the normal Xray using minimal computing resources that achieve better results compared to traditional models.

# Importing packages


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Activation, ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import sigmoid, softmax, relu
from tensorflow.keras.applications.vgg16 import VGG16

check the number of folders in the parcicular folder

In [2]:
import glob
from glob import glob
glob("/content/drive/MyDrive/CNN Projects/chest-xray-multi-class/train/*")

['/content/drive/MyDrive/CNN Projects/chest-xray-multi-class/train/NORMAL',
 '/content/drive/MyDrive/CNN Projects/chest-xray-multi-class/train/PNEUMONIA',
 '/content/drive/MyDrive/CNN Projects/chest-xray-multi-class/train/COVID19']

In [3]:
glob("/content/drive/MyDrive/CNN Projects/chest-xray-multi-class/test/*")

['/content/drive/MyDrive/CNN Projects/chest-xray-multi-class/test/COVID19',
 '/content/drive/MyDrive/CNN Projects/chest-xray-multi-class/test/NORMAL',
 '/content/drive/MyDrive/CNN Projects/chest-xray-multi-class/test/PNEUMONIA']

# Code

In [4]:
class VGG16_multi:

    def __init__(self, path):
        self.path = path
        self.train_path = "/content/drive/MyDrive/CNN Projects/chest-xray-multi-class/train"
        self.test_path = "/content/drive/MyDrive/CNN Projects/chest-xray-multi-class/test"
        self.height,self.width= [200, 200]
        self.label = ["NORMAL", "PNEUMONIA","COVID19" ]

    def image_augmentation(self):
        try:
            preprocess_train_data = ImageDataGenerator(rescale = 1.0/255.0, shear_range=0.2, horizontal_flip = True,
                                                       zoom_range = 0.2)
            preprocess_test_data = ImageDataGenerator(rescale = 1.0/255.0)
            train_data, test_data = self.preprocess(preprocess_train_data, preprocess_test_data)
            model = self.architecture(train_data, test_data)
            self.validation(model)

        except Exception as e:
            print(f"The error is in image_augmentation {e.__str__()}")

    def preprocess(self, preprocess_train_data, preprocess_test_data):
        try:
            train_data = preprocess_train_data.flow_from_directory(self.train_path, target_size=(self.height, self.width),
                                                                   class_mode="categorical", classes=self.label, batch_size=22)
            test_data = preprocess_test_data.flow_from_directory(self.test_path, target_size=(self.height, self.width),
                                                                 class_mode="categorical", classes=self.label, batch_size=22)
            return train_data, test_data
        except Exception as e:
            print(f"The error is in preprocess{e.__str__()}")

    def architecture(self, train_data, test_data):
        try:
            vgg16 = VGG16(input_shape = (self.height, self.width, 3), weights = 'imagenet', include_top=False)
            for layers in vgg16.layers:
                layers.trainable = False
            x = Flatten()(vgg16.output) #1D array
            predict = Dense(3, activation="softmax")(x) # 2 or more classes
            self.model = Model(inputs = vgg16.inputs, outputs = predict)
            self.model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
            self.model.fit(train_data, validation_data=test_data, epochs=10, steps_per_epoch=1, validation_steps=2)
            return self.model
        except Exception as e:
            print(f"The error in architecture{e.__str__()}")

    def validation(self, model):
        try:
            pic = cv2.imread("/content/drive/MyDrive/CNN Projects/chest-xray-multi-class/pred/N(10).jpg")
            print(pic.shape)
            pic = cv2.resize(pic, (self.height, self.width))
            pic = (np.array(pic))/255
            pic = np.expand_dims(pic,axis= 0)
            result = model.predict(pic)
            sol = np.argmax(result , axis = 1) #
            print(sol[0])
            print(f"the image belongs to category:{self.label[sol[0]]}")
        except Exception as e:
            print(f"The error in validation {e.__str__()}")


if __name__== "__main__":
    try:
        obj = VGG16_multi("/content/drive/MyDrive/CNN Projects/chest-xray-multi-class")
        obj.image_augmentation()
    except Exception as e:
        print(f"The error is in main {e.__str__()}")

Found 4735 images belonging to 3 classes.
Found 1282 images belonging to 3 classes.
Epoch 1/10
1/1 [==============================] - 23s 23s/step - loss: 1.0448 - accuracy: 0.3182 - val_loss: 2.0251 - val_accuracy: 0.5455
Epoch 2/10
1/1 [==============================] - 13s 13s/step - loss: 2.1365 - accuracy: 0.5000 - val_loss: 1.0985 - val_accuracy: 0.6818
Epoch 3/10
1/1 [==============================] - 13s 13s/step - loss: 1.0802 - accuracy: 0.6364 - val_loss: 1.1610 - val_accuracy: 0.6591
Epoch 4/10
1/1 [==============================] - 14s 14s/step - loss: 1.7678 - accuracy: 0.4091 - val_loss: 0.7701 - val_accuracy: 0.7500
Epoch 5/10
1/1 [==============================] - 11s 11s/step - loss: 0.8783 - accuracy: 0.5909 - val_loss: 0.8722 - val_accuracy: 0.6818
Epoch 6/10
1/1 [==============================] - 11s 11s/step - loss: 1.0429 - accuracy: 0.6818 - val_loss: 0.5365 - val_accuracy: 0.7955
Epoch 7/10
1/1 [==============================] - 11s 11s/step - loss: 0.7913 - ac